# Top 250 Football transfers from 2000 to 2018

In [88]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool, ColorBar
from bokeh.layouts import row, column, gridplot
from bokeh.transform import factor_cmap

In [3]:
tr = pd.read_csv('top250-00-19.csv')

In [4]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 10 columns):
Name            4700 non-null object
Position        4700 non-null object
Age             4700 non-null int64
Team_from       4700 non-null object
League_from     4700 non-null object
Team_to         4700 non-null object
League_to       4700 non-null object
Season          4700 non-null object
Market_value    3440 non-null float64
Transfer_fee    4700 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 367.3+ KB


In [138]:
tr.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [5]:
tr['transfer_fee_M']= tr['Transfer_fee'].map(lambda x : str(x/1000000)+' M')

In [6]:
tr['transfer_fee_M'].head()

0     60.0 M
1    56.81 M
2     40.0 M
3    36.15 M
4     34.5 M
Name: transfer_fee_M, dtype: object

In [7]:
tr['Year']= tr['Season'].map(lambda x : x.split('-')[0])
tr['Year'] = tr['Year'].astype(int)

In [8]:
tr = tr.sort_values('Transfer_fee', ascending=False)

In [9]:
tr_yearly_10 = tr.groupby('Year').head(10).reset_index(drop=True)
tr_max_100 = tr.head(100).reset_index(drop=True)
tr_max_20 = tr.head(20).reset_index(drop=True)

In [82]:
source1 = ColumnDataSource(tr_max_20)
p1 = figure(plot_width=800, plot_height=500,x_axis_label='Year', 
            y_axis_label='Transfer Fee', tools='box_select, lasso_select',)
p1.circle('Year', 'Transfer_fee', size=8, source=source1, 
            selection_color='red',
            nonselection_fill_alpha=0.2, nonselection_fill_color='grey',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')

bokeh.models.renderers.GlyphRenderer(
    id='78fb409e-f2f0-4074-a13b-0117e02f6dd1',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='e092222f-2903-43da-867c-c676fe0b3197',
        callback=None,
        data={'Season': ['2017-2018',
          '2018-2019',
          '2017-2018',
          '2018-2019',
          '2017-2018',
          '2016-2017',
          '2013-2014',
          '2009-2010',
          '2016-2017',
          '2013-2014',
          '2017-2018',
          '2014-2015',
          '2017-2018',
          '2001-2002',
          '2015-2016',
          '2014-2015',
          '2014-2015',
          '2018-2019',
          '2009-2010',
          '2018-2019'],
         'Market_value': {'__ndarray__': 'AAAAAITXl0EAAAAAOJycQQAAAAAqdZVBAAAAAITXl0EAAAAApHh/QQAAAAB2sJBBAAAAAJL+jkEAAAAAOJyMQQAAAACS/o5BAAAAAITXh0EAAAAAhNeHQQAAAACoy4hBAAAAADicfEEAAAAAAAD4fwAAAAAqdYVBAAAAADicjEEAAAAAhNeHQQAAAACS/o5BAAAAACp1hUEAAAAAhNeHQQ==',
          'shape': (20,),
          'dtype': 'float64'},
         'Age': [25,
          19,
          25,
          33,
          20,
          23,
          24,
          24,
          28,
          21,
          24,
          27,
          26,
          29,
          24,
          23,
          26,
          22,
          27,
          27],
         'Transfer_fee': [222000000,
          135000000,
          125000000,
          117000000,
          115000000,
          105000000,
          101000000,
          94000000,
          90000000,
          88200000,
          84700000,
          81720000,
          78800000,
          77500000,
          76000000,
          75000000,
          75000000,
          70000000,
          69500000,
          67800000],
         'Year': {'__ndarray__': '4QcAAOIHAADhBwAA4gcAAOEHAADgBwAA3QcAANkHAADgBwAA3QcAAOEHAADeBwAA4QcAANEHAADfBwAA3gcAAN4HAADiBwAA2QcAAOIHAAA=',
          'shape': (20,),
          'dtype': 'int32'},
         'Team_from': ['FC Barcelona',
          'Monaco',
          'Liverpool',
          'Real Madrid',
          'Bor. Dortmund',
          'Juventus',
          'Spurs',
          'Man Utd',
          'SSC Napoli',
          'Santos FC',
          'Everton',
          'Liverpool',
          'Southampton',
          'Juventus',
          'VfL Wolfsburg',
          'Monaco',
          'Real Madrid',
          'Monaco',
          'Inter',
          'Leicester'],
         'League_from': ['LaLiga',
          'Ligue 1',
          'Premier League',
          'LaLiga',
          '1.Bundesliga',
          'Serie A',
          'Premier League',
          'Premier League',
          'Serie A',
          'Série A',
          'Premier League',
          'Premier League',
          'Premier League',
          'Serie A',
          '1.Bundesliga',
          'Ligue 1',
          'LaLiga',
          'Ligue 1',
          'Serie A',
          'Premier League'],
         'Team_to': ['Paris SG',
          'Paris SG',
          'FC Barcelona',
          'Juventus',
          'FC Barcelona',
          'Man Utd',
          'Real Madrid',
          'Real Madrid',
          'Juventus',
          'FC Barcelona',
          'Man Utd',
          'FC Barcelona',
          'Liverpool',
          'Real Madrid',
          'Man City',
          'Real Madrid',
          'Man Utd',
          'Atlético Madrid',
          'FC Barcelona',
          'Man City'],
         'Name': ['Neymar',
          'Kylian Mbappé',
          'Philippe Coutinho',
          'Cristiano Ronaldo',
          'Ousmane Dembélé',
          'Paul Pogba',
          'Gareth Bale',
          'Cristiano Ronaldo',
          'Gonzalo Higuaín',
          'Neymar',
          'Romelu Lukaku',
          'Luis Suárez',
          'Virgil van Dijk',
          'Zinédine Zidane',
          'Kevin De Bruyne',
          'James Rodríguez',
          'Ángel Di María',
          'Thomas Lemar',
          'Zlatan Ibrahimovic',
          'Riyad Mahrez'],
         'index': [0,
          1,
          2,
          3,
          4,


In [83]:
source2 = ColumnDataSource(tr_yearly_10)
p2 = figure(plot_width=800, plot_height=500,
            x_axis_label='Year', y_axis_label='Transfer Fee', tools='box_select, lasso_select',)
p2.circle('Year', 'Transfer_fee', size=8, source=source2, 
            selection_color='red',
            nonselection_fill_alpha=0.2, nonselection_fill_color='grey',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')

bokeh.models.renderers.GlyphRenderer(
    id='d8341390-b27e-42d8-a74a-7074180f96ff',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='eaf7bba3-14a2-4e4a-98b6-9d348d50a4ba',
        callback=None,
        data={'Season': ['2017-2018',
          '2018-2019',
          '2017-2018',
          '2018-2019',
          '2017-2018',
          '2016-2017',
          '2013-2014',
          '2009-2010',
          '2016-2017',
          '2013-2014',
          '2017-2018',
          '2014-2015',
          '2017-2018',
          '2001-2002',
          '2015-2016',
          '2014-2015',
          '2014-2015',
          '2018-2019',
          '2009-2010',
          '2018-2019',
          '2017-2018',
          '2017-2018',
          '2017-2018',
          '2009-2010',
          '2013-2014',
          '2017-2018',
          '2015-2016',
          '2015-2016',
          '2018-2019',
          '2015-2016',
          '2018-2019',
          '2016-2017',
          '2000-2001',
          '2018-2019',
          '2010-2011',
          '2017-2018',
          '2018-2019',
          '2000-2001',
          '2016-2017',
          '2016-2017',
          '2001-2002',
          '2016-2017',
          '2015-2016',
          '2014-2015',
          '2001-2002',
          '2013-2014',
          '2015-2016',
          '2002-2003',
          '2016-2017',
          '2018-2019',
          '2018-2019',
          '2013-2014',
          '2002-2003',
          '2013-2014',
          '2015-2016',
          '2006-2007',
          '2008-2009',
          '2013-2014',
          '2015-2016',
          '2001-2002',
          '2014-2015',
          '2012-2013',
          '2001-2002',
          '2011-2012',
          '2016-2017',
          '2015-2016',
          '2001-2002',
          '2016-2017',
          '2010-2011',
          '2015-2016',
          '2016-2017',
          '2011-2012',
          '2010-2011',
          '2012-2013',
          '2012-2013',
          '2012-2013',
          '2012-2013',
          '2013-2014',
          '2011-2012',
          '2000-2001',
          '2013-2014',
          '2001-2002',
          '2004-2005',
          '2005-2006',
          '2008-2009',
          '2014-2015',
          '2007-2008',
          '2014-2015',
          '2003-2004',
          '2010-2011',
          '2004-2005',
          '2001-2002',
          '2013-2014',
          '2000-2001',
          '2014-2015',
          '2002-2003',
          '2001-2002',
          '2008-2009',
          '2014-2015',
          '2009-2010',
          '2007-2008',
          '2012-2013',
          '2012-2013',
          '2000-2001',
          '2009-2010',
          '2011-2012',
          '2010-2011',
          '2014-2015',
          '2003-2004',
          '2012-2013',
          '2007-2008',
          '2005-2006',
          '2012-2013',
          '2002-2003',
          '2012-2013',
          '2008-2009',
          '2009-2010',
          '2004-2005',
          '2011-2012',
          '2011-2012',
          '2007-2008',
          '2010-2011',
          '2007-2008',
          '2010-2011',
          '2009-2010',
          '2009-2010',
          '2010-2011',
          '2001-2002',
          '2004-2005',
          '2009-2010',
          '2011-2012',
          '2006-2007',
          '2009-2010',
          '2007-2008',
          '2008-2009',
          '2004-2005',
          '2011-2012',
          '2005-2006',
          '2011-2012',
          '2010-2011',
          '2000-2001',
          '2006-2007',
          '2003-2004',
          '2011-2012',
          '2007-2008',
          '2006-2007',
          '2005-2006',
          '2000-2001',
          '2000-2001',
          '2010-2011',
          '2005-2006',
          '2007-2008',
          '2004-2005',
          '2008-2009',
          '2006-2007',
          '2007-2008',
          '2008-2009',
          '2003-2004',
          '2007-2008',
          '2005-2006',
          '2008-2009',
          '2008-2009',
          '2003-2004',
          '2000-2001',

In [120]:
from bokeh.palettes import Category20_12
from bokeh.models import FactorRange, ColorBar
from bokeh.transform import linear_cmap
from math import pi

group = tr_max_100.head(10).sort_values('Year')
source3= ColumnDataSource(group)
p3 = figure(plot_width=800, plot_height=500, title="Most Expensive 100 Transfers",
           x_range=FactorRange(factors=player_list), toolbar_location=None, tools="")

p3.xgrid.grid_line_color = None
p3.xaxis.axis_label = "Most Expensive Players"
p3.xaxis.major_label_orientation = 1.2
p3.xaxis.major_label_text_color = "SeaGreen"
p3.xaxis.major_label_orientation = pi/4
p3.xaxis.major_label_text_font_size = '12pt'

player_list = group.Name.unique().tolist()

index_cmap = factor_cmap('Position', palette=Category20_12, 
                         factors=player_list)

#Use the field name of the column source
mapper = linear_cmap(field_name='Transfer_fee', palette=Category20_12 ,
                     low=min(tr_max_100['Transfer_fee']),high=max(tr_max_100['Transfer_fee']))

p3.circle(x='Name', y='Transfer_fee', source=source3,
       line_color=mapper, fill_color=mapper, size = 20,
       hover_line_color="darkgrey", hover_fill_color=index_cmap)



hover = HoverTool(tooltips = [('Player', '@Name'), ('From (Team)', '@Team_from'),
                             ('To (Team)', '@Team_to'), ('Transfer Fee', '@transfer_fee_M'), 
                              ('Year', '@Year')])

#color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
#p3.add_layout(color_bar, 'right')

p3.add_tools(hover)

#outline_border
p3.outline_line_width = 7
p3.outline_line_alpha = 0.3
p3.outline_line_color = "navy"

show(p3)

In [118]:
# Import Panel from bokeh.models.widgets
from bokeh.models.widgets import Panel

# Create tab1 from plot p1: tab1
tab1 = Panel(child=p1, title='Maximum 20 Transfer Fee')

# Create tab2 from plot p2: tab2
tab2 = Panel(child=p2, title='Max 10 Transfer Fee (Yearly)')

# Create tab2 from plot p2: tab2
tab3 = Panel(child=p3, title='Max 10 Transfer Fee')

# Import Tabs from bokeh.models.widgets
from bokeh.models.widgets import Tabs

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2, tab3])

# Specify the name of the output_file and show the result
output_notebook()
show(layout)

Loading BokehJS ...

In [1]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

In [95]:
# Add circle glyphs to the plot
plot.circle(x='x', y='y', fill_alpha=0.8, source=source)

# Set the x-axis label
plot.xaxis.axis_label ='Age'

# Set the y-axis label
plot.yaxis.axis_label = 'Transfer Fee'

plot.add_tools(hover)

# Add the plot to the current document and add a title
curdoc().add_root(plot)
curdoc().title = 'Transfers'

#output_notebook()
#show(plot)

In [123]:
# Make the ColumnDataSource: source
tr_plot = tr_yearly_10

data = {
    'x'        : tr_plot[tr_plot['Year']==2000].Age,
    'y'        : tr_plot[tr_plot['Year']==2000].Transfer_fee,
    'position' : tr_plot[tr_plot['Year']==2000].Position,
    'league_from'     : tr_plot[tr_plot['Year']==2000].League_from,
    'league_to': tr_plot[tr_plot['Year']==2000].League_to,
    'name'        : tr_plot[tr_plot['Year']==2000].Name,
    'team_to' : tr_plot[tr_plot['Year']==2000].Team_to,
    'team_from'     : tr_plot[tr_plot['Year']==2000].Team_from,
    'transfer_fee_M': tr_plot[tr_plot['Year']==2000].transfer_fee_M,    
}
source = ColumnDataSource(data=data)

# Create the figure: plot
plot = figure(title='Transfers in 2000', plot_width=700, plot_height=400,x_range=(xmin, xmax), y_range=(ymin, ymax))
#plot = figure(title='Transfers in 2000', plot_width=700)
hover = HoverTool(tooltips = [('Player', '@name'), ('From (Team)', '@team_from'),
                             ('To (Team)', '@team_to'), ('Transfer Fee', '@transfer_fee_M')])

# Save the minimum and maximum values of the fertility column: xmin, xmax
xmin, xmax = min(tr_plot.Age), max(tr_plot.Age)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(tr_plot.Transfer_fee), max(tr_plot.Transfer_fee)

In [124]:
# Create the figure: plot
plot = figure(title='Transfers in 2000', plot_width=700, plot_height=400,x_range=(xmin, xmax), y_range=(ymin, ymax))
#plot = figure(title='Transfers in 2000', plot_width=700)

# Make a list of the unique values from the region column: regions_list
league_list = tr_plot.League_to.unique().tolist()

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import viridis, Spectral7, Set2_7

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=league_list, palette=Set2_7)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=1, source=source, size=10,
            color=dict(field='league_to', transform=color_mapper), legend='league_to',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')

callback = CustomJS(args=dict(source=source), code="""
    data ={
        'x'        : tr_plot[tr_plot['Year']==yr].Age,
        'y'        : tr_plot[tr_plot['Year']==yr].Transfer_fee,
        'position' : tr_plot[tr_plot['Year']==yr].Position,
        'league_from'     : tr_plot[tr_plot['Year']==yr].League_from,
        'league_to': tr_plot[tr_plot['Year']==yr].League_to,
        'name'        : tr_plot[tr_plot['Year']==yr].Name,
        'team_to' : tr_plot[tr_plot['Year']==yr].Team_to,
        'team_from'     : tr_plot[tr_plot['Year']==yr].Team_from,
        'transfer_fee_M': tr_plot[tr_plot['Year']==yr].transfer_fee_M,
        }
     source.change.emit();
""")

year_slider = Slider(start=2000, end=2018, value=2000, step=1,
                    title="Year", callback=callback)
callback.args["yr"] = year_slider

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'

plot.add_tools(hover)

layout = row(plot,
    widgetbox(year_slider),
)

# Add the plot to the current document and add the title
curdoc().add_root(layout)
curdoc().title = 'Transfers'

output_file('Transfers.html')
show(layout)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: League_to [renderer: GlyphRenderer(id='937c5761-b742-4bff-9e53-8f3bf48aa996', ...)]


In [ ]:
# Create the figure: plot
plot = figure(title='Transfers in 2000', plot_width=700, plot_height=400,x_range=(xmin, xmax), y_range=(ymin, ymax))
#plot = figure(title='Transfers in 2000', plot_width=700)

# Make a list of the unique values from the region column: regions_list
league_list = tr_plot.League_to.unique().tolist()

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import viridis, Spectral7, Set2_7

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=league_list, palette=Set2_7)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=1, source=source, size=10,
            color=dict(field='league_to', transform=color_mapper), legend='league_to',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')



# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'

plot.add_tools(hover)

# Add the plot to the current document and add the title
curdoc().add_root(plot)
curdoc().title = 'Transfers'

output_file('Transfers.html')
show(plot)

In [157]:
del plot

In [173]:
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import layout

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import viridis, Spectral7, Set2_7

tr_plot = tr_yearly_10

# Save the minimum and maximum values of the fertility column: xmin, xmax
xmin, xmax = min(tr_plot.Age), max(tr_plot.Age)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(tr_plot.Transfer_fee), max(tr_plot.Transfer_fee)

yr=2000
data_init = {
        'x'        : tr_plot[tr_plot['Year']==yr].Age,
        'y'        : tr_plot[tr_plot['Year']==yr].Transfer_fee,
        'position' : tr_plot[tr_plot['Year']==yr].Position,
        'league_from'     : tr_plot[tr_plot['Year']==yr].League_from,
        'league_to': tr_plot[tr_plot['Year']==yr].League_to,
        'name'        : tr_plot[tr_plot['Year']==yr].Name,
        'team_to' : tr_plot[tr_plot['Year']==yr].Team_to,
        'team_from'     : tr_plot[tr_plot['Year']==yr].Team_from,
        'transfer_fee_M': tr_plot[tr_plot['Year']==yr].transfer_fee_M,
        }
source = ColumnDataSource(data=data_init)

# Create the figure: plot
#plot = figure(title='Transfers in 2000', plot_width=700, plot_height=400,x_range=(xmin, xmax), y_range=(ymin, ymax))
plot = figure(title=str(year_slider.value), plot_width=700)

hover = HoverTool(tooltips = [('Player', '@name'), ('From (Team)', '@team_from'),
                             ('To (Team)', '@team_to'), ('Transfer Fee', '@transfer_fee_M')])

def callback (attr, old, new):
    new_yr = new
    new1 = ColumnDataSource(data={
        'x'        : tr_plot[tr_plot['Year']==new_yr].Age,
        'y'        : tr_plot[tr_plot['Year']==new_yr].Transfer_fee,
        'position' : tr_plot[tr_plot['Year']==new_yr].Position,
        'league_from'     : tr_plot[tr_plot['Year']==new_yr].League_from,
        'league_to': tr_plot[tr_plot['Year']==new_yr].League_to,
        'name'        : tr_plot[tr_plot['Year']==new_yr].Name,
        'team_to' : tr_plot[tr_plot['Year']==new_yr].Team_to,
        'team_from'     : tr_plot[tr_plot['Year']==new_yr].Team_from,
        'transfer_fee_M': tr_plot[tr_plot['Year']==new_yr].transfer_fee_M,
        })
    source.data=new1.data
    
year_slider = Slider(start=2000, end=2018, value=2015, step=1,
                    title="Year") 

year_slider.on_change('value', callback)

# Make a list of the unique values from the region column: regions_list
league_list = tr_plot.League_to.unique().tolist()

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=league_list, palette=Set2_7)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=1, source=source, size=10,
            color=dict(field='league_to', transform=color_mapper), legend='league_to',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')
plot.legend.location = 'top_right'

plot.add_tools(hover)

layout = column(widgetbox(year_slider), plot)

curdoc().add_root(layout)

output_notebook()
show(layout)



Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: League_to [renderer: GlyphRenderer(id='937c5761-b742-4bff-9e53-8f3bf48aa996', ...)]


In [175]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  9 23:03:16 2018

@author: Ghema
"""

import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_file
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.layouts import widgetbox, column
from bokeh.io import curdoc
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Set2_7
from bokeh.models import Slider
tr = pd.read_csv('top250-00-19.csv')

tr['transfer_fee_M']= tr['Transfer_fee'].map(lambda x : str(x/1000000)+' M')

tr['Year']= tr['Season'].map(lambda x : x.split('-')[0])
tr['Year'] = tr['Year'].astype(int)

tr = tr.sort_values('Transfer_fee', ascending=False)

tr_yearly_10 = tr.groupby('Year').head(10).reset_index(drop=True)
tr_max_100 = tr.head(100).reset_index(drop=True)
tr_max_20 = tr.head(20).reset_index(drop=True)


tr_plot = tr_yearly_10

# Save the minimum and maximum values of the fertility column: xmin, xmax
xmin, xmax = min(tr_plot.Age), max(tr_plot.Age)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(tr_plot.Transfer_fee), max(tr_plot.Transfer_fee)

yr=2000
data_init = {
        'x'        : tr_plot[tr_plot['Year']==yr].Age,
        'y'        : tr_plot[tr_plot['Year']==yr].Transfer_fee,
        'position' : tr_plot[tr_plot['Year']==yr].Position,
        'league_from'     : tr_plot[tr_plot['Year']==yr].League_from,
        'league_to': tr_plot[tr_plot['Year']==yr].League_to,
        'name'        : tr_plot[tr_plot['Year']==yr].Name,
        'team_to' : tr_plot[tr_plot['Year']==yr].Team_to,
        'team_from'     : tr_plot[tr_plot['Year']==yr].Team_from,
        'transfer_fee_M': tr_plot[tr_plot['Year']==yr].transfer_fee_M,
        }
source = ColumnDataSource(data=data_init)

hover = HoverTool(tooltips = [('Player', '@name'), ('From (Team)', '@team_from'),
                             ('To (Team)', '@team_to'), ('Transfer Fee', '@transfer_fee_M')])



plot = figure(title='Transfers', plot_width=700, plot_height = 450 )

# Make a color mapper: color_mapper
league_list = tr_plot.League_to.unique().tolist()
color_mapper = CategoricalColorMapper(factors=league_list, palette=Set2_7)

c = plot.circle(x='x', y='y', fill_alpha=1, source=source, size=10,
            color=dict(field='league_to', transform=color_mapper), legend='league_to',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')

plot.legend.location = 'top_right'

year_slider = Slider(start=2000, end=2018, value=2000, step=1, title="Year") 

def callback (attr, old, new):
    new_yr = new
    new1 = {
        'x'        : tr_plot[tr_plot['Year']==new_yr].Age,
        'y'        : tr_plot[tr_plot['Year']==new_yr].Transfer_fee,
        'position' : tr_plot[tr_plot['Year']==new_yr].Position,
        'league_from'     : tr_plot[tr_plot['Year']==new_yr].League_from,
        'league_to': tr_plot[tr_plot['Year']==new_yr].League_to,
        'name'        : tr_plot[tr_plot['Year']==new_yr].Name,
        'team_to' : tr_plot[tr_plot['Year']==new_yr].Team_to,
        'team_from'     : tr_plot[tr_plot['Year']==new_yr].Team_from,
        'transfer_fee_M': tr_plot[tr_plot['Year']==new_yr].transfer_fee_M
        }
    source.data=new1
    
year_slider.on_change('value', callback)

layout = column(widgetbox(year_slider), plot)

curdoc().add_root(layout)

#output_file('Transfer.html')
show(layout)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: League_to [renderer: GlyphRenderer(id='937c5761-b742-4bff-9e53-8f3bf48aa996', ...)]


In [185]:
del plot

In [208]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  9 23:03:16 2018

@author: Ghema
"""

import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_file
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool,CDSView, BooleanFilter
from bokeh.layouts import widgetbox, column
from bokeh.io import curdoc
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Set2_7
from bokeh.models import Slider
tr = pd.read_csv('top250-00-19.csv')

tr['transfer_fee_M']= tr['Transfer_fee'].map(lambda x : str(x/1000000)+' M')

tr['Year']= tr['Season'].map(lambda x : x.split('-')[0])
tr['Year'] = tr['Year'].astype(int)

tr = tr.sort_values('Transfer_fee', ascending=False)

tr_yearly_10 = tr.groupby('Year').head(10).reset_index(drop=True)
tr_max_100 = tr.head(100).reset_index(drop=True)
tr_max_20 = tr.head(20).reset_index(drop=True)


tr_plot = tr_yearly_10

# Save the minimum and maximum values of the fertility column: xmin, xmax
xmin, xmax = min(tr_plot.Age), max(tr_plot.Age)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(tr_plot.Transfer_fee), max(tr_plot.Transfer_fee)

data = {
        'x'        : tr_plot.Age,
        'y'        : tr_plot.Transfer_fee,
        'position' : tr_plot.Position,
        'league_from'     : tr_plot.League_from,
        'league_to': tr_plot.League_to,
        'name'        : tr_plot.Name,
        'team_to' : tr_plot.Team_to,
        'team_from'     : tr_plot.Team_from,
        'transfer_fee_M': tr_plot.transfer_fee_M,
        'year' : tr_plot.Year,
        }
source = ColumnDataSource(data=data)
booleans = [True if year_val == 2012 else False for year_val in tr_plot.Year]
filter1 = ['tr_plot.Year==2017']
view = CDSView(source=source, filters=[BooleanFilter(booleans)])

hover = HoverTool(tooltips = [('Player', '@name'), ('From (Team)', '@team_from'),
                             ('To (Team)', '@team_to'), ('Transfer Fee', '@transfer_fee_M'), 
                              ('Year', '@year')])

plot = figure(title='Transfers', plot_width=700, plot_height = 450 )

# Make a color mapper: color_mapper
league_list = tr_plot.League_to.unique().tolist()
color_mapper = CategoricalColorMapper(factors=league_list, palette=Set2_7)

c = plot.circle(x='x', y='y', fill_alpha=1, source=source, view=view, size=10,
            color=dict(field='league_to', transform=color_mapper), legend='league_to',
            hover_fill_color='firebrick', hover_alpha=1,
            hover_line_color='red')

plot.legend.location = 'top_right'

plot.add_tools(hover)

year_slider = Slider(start=2000, end=2018, value=2000, step=1, title="Year") 

def callback (attr, old, new):
    new_yr = new
    source.data=new1
    
year_slider.on_change('value', callback)

layout = column(widgetbox(year_slider), plot)

#output_file('Transfer.html')
show(layout)

ValueError: expected an element of List(Instance(Filter)), got seq with invalid items [['tr_plot.Year==2017']]

In [197]:
booleans.count(False)

180

In [ ]:
from bokeh.sampledata.